In [4]:
# Cool tests calling these functions are below the function definitions
import os

import sqlite3
import pandas as pd

In [5]:
# Path to the root directory of datasci
dthon_path = os.path.join(os.path.expanduser('~'), 'Projects','datasci')
dthon_db = os.path.join(dthon_path, "melbourne-datathon","data","medi.db")

In [6]:
# select a patient based on a patient ID and determine whether transactions
# are associated with a specified illness (is_{illness}_medication)
def patient_transactions_illness(patient_id, illness, conn):
    sql = """
        SELECT
            t2.Patient_ID,
            t2.'Store_ID',
            t2.'Prescriber_ID',
            t2.'Drug_ID',
            t2.'Prescription_Week',
            t2.'Dispense_Week',
            t2.'Drug_Code',
            t2.'NHS_Code',
            t2.'PatientPrice_Amt', 
            t2.'WholeSalePrice_Amt',
            t2.'GovernmentReclaim_Amt', 
            t2.'StreamlinedApproval_Code', 
            -- t2.'ChemistListPrice',
            p.year_of_birth,
            p.postcode,
            p.gender,
            EXISTS(
                SELECT 1
                WHERE t2.Drug_ID IN (
                    SELECT c2.MasterProductID
                    FROM ChronicIllness_LookUp c2
                    WHERE c2.ChronicIllness = '{1}'
                )
            ) AS is_{1}_medication
        FROM TRANSACTIONS t2
        JOIN patients p ON t2.Patient_ID = p.Patient_ID
        WHERE t2.Patient_ID = {0};
    """.format(patient_id, illness)

    return pd.read_sql_query(sql, conn)

In [21]:
# grabs no. of random patients defined by LIMIT below and can be handled by my memory
def bulk_patient_transactions_illness(num, illness, conn):
    sql = """
        SELECT
            t2.Patient_ID,
            t2.'Store_ID',
            t2.'Prescriber_ID',
            t2.'Drug_ID',
            t2.'Prescription_Week',
            t2.'Dispense_Week',
            t2.'Drug_Code',
            t2.'NHS_Code',
            t2.'PatientPrice_Amt', 
            t2.'WholeSalePrice_Amt',
            t2.'GovernmentReclaim_Amt', 
            t2.'StreamlinedApproval_Code', 
            -- t2.'ChemistListPrice',
            p.year_of_birth,
            p.postcode,
            p.gender,
            EXISTS(
                SELECT 1
                WHERE t2.Drug_ID IN (
                    SELECT c2.MasterProductID
                    FROM ChronicIllness_LookUp c2
                    WHERE c2.ChronicIllness = '{1}'
                )
            ) AS is_{1}_medication

        FROM TRANSACTIONS t2
        JOIN patients p ON t2.Patient_ID = p.Patient_ID
        WHERE t2.Patient_ID IN (
            SELECT ic.Patient_ID FROM implied_condition ic
            WHERE ic.ChronicIllness = '{1}'
            GROUP BY ic.Patient_ID -- Yeilds all patients that have ever taken db medi
            ORDER BY RANDOM()
            LIMIT {0} -- grab only this many random people
        );
    """.format(num, illness)

    return pd.read_sql_query(sql, conn)


In [8]:
# Run this first to create database connection
conn = sqlite3.connect(dthon_db)

In [10]:
# This dataframe is just patient 2, takes seconds
%time df = patient_transactions_illness(2, 'Diabetes',conn)

CPU times: user 5.07 s, sys: 404 ms, total: 5.47 s
Wall time: 5.52 s


In [11]:
df.head()

,Patient_ID,Store_ID,Prescriber_ID,Drug_ID,Prescription_Week,Dispense_Week,Drug_Code,NHS_Code,PatientPrice_Amt,WholeSalePrice_Amt,GovernmentReclaim_Amt,StreamlinedApproval_Code,year_of_birth,postcode,gender,is_Diabetes_medication
0,2,65,10206,3228,2010-12-05,2011-01-09,EZET1,8757X,5.6,58.68,65.37,,1935,3042,F,0
1,2,65,10206,3228,2010-12-05,2011-02-13,EZET1,8757X,5.6,58.68,65.37,,1935,3042,F,0
2,2,65,10206,3228,2010-12-05,2011-03-20,EZET1,8757X,5.6,58.68,65.37,,1935,3042,F,0
3,2,65,10206,4540,2011-01-23,2011-03-20,KARV4,8404H,5.6,18.22,21.77,,1935,3042,F,0
4,2,65,0,3863,2010-08-08,2011-04-17,GLYA1,2449X,5.6,5.86,7.56,,1935,3042,F,1


In [17]:
#Control, 10 patients, old implementation of implied condition in the form of a nasty query (gone now)
%time dflc = bulk_patient_transactions_illness(10, 'Diabetes',conn)

CPU times: user 34.2 s, sys: 22.1 s, total: 56.3 s
Wall time: 3min 54s


In [18]:
dflc.head()

,Patient_ID,Store_ID,Prescriber_ID,Drug_ID,Prescription_Week,Dispense_Week,Drug_Code,NHS_Code,PatientPrice_Amt,WholeSalePrice_Amt,GovernmentReclaim_Amt,StreamlinedApproval_Code,year_of_birth,postcode,gender,is_Diabetes_medication
0,78040,2800,12381,862,2010-07-11,2011-01-23,AVAP2,8405J,5.6,22.64,26.86,,1900,3071,F,0
1,78040,2800,12381,862,2010-06-06,2011-01-30,AVAP2,8405J,5.6,22.64,26.86,,1900,3071,F,0
2,78040,2800,12381,6494,2010-12-05,2011-01-30,PANX1,8784H,5.6,4.96,6.52,,1900,3071,F,0
3,78040,2800,12381,862,2010-06-06,2011-02-20,AVAP2,8405J,5.6,22.64,26.86,,1900,3071,F,0
4,78040,2800,12381,3200,2011-02-27,2011-03-06,EVIS1,8363E,5.6,46.77,52.27,,1900,3071,F,0


In [19]:
#Control, but grabbing 1000 patients, shows that increasing sample size doesnt increase time substantially (gone now)
%time dflc = bulk_patient_transactions_illness(1000, 'Diabetes',conn)

CPU times: user 47.7 s, sys: 21.4 s, total: 1min 9s
Wall time: 3min 54s


In [22]:
# now using implied condition database table to speed things up (No indexes on implied_condition table yet)
%time dfl_optimised = bulk_patient_transactions_illness(1000, 'Diabetes',conn)

CPU times: user 34.1 s, sys: 984 ms, total: 35 s
Wall time: 36.2 s


In [25]:
dfl_optimised.head()

,Patient_ID,Store_ID,Prescriber_ID,Drug_ID,Prescription_Week,Dispense_Week,Drug_Code,NHS_Code,PatientPrice_Amt,WholeSalePrice_Amt,GovernmentReclaim_Amt,StreamlinedApproval_Code,year_of_birth,postcode,gender,is_Diabetes_medication
0,1365,734,50140,6523,2010-10-03,2011-01-09,PARI3,8508T,5.6,27.27,32.18,,1900,2470,F,0
1,1365,734,50140,2119,2010-10-03,2011-01-16,CRES3,9045C,5.6,103.43,114.59,,1900,2470,F,0
2,1365,734,50140,6523,2010-10-03,2011-02-13,PARI3,8508T,5.6,26.18,30.93,,1900,2470,F,0
3,1365,734,50140,2119,2010-10-03,2011-02-20,CRES3,9045C,5.6,103.43,114.59,,1900,2470,F,0
4,1365,734,50140,6523,2010-10-03,2011-03-13,PARI3,8508T,5.6,26.18,30.93,,1900,2470,F,0


In [26]:
# Much better, now try 10000?
%time dfll_optimised = bulk_patient_transactions_illness(10000, 'Diabetes',conn)

CPU times: user 52.5 s, sys: 3.06 s, total: 55.5 s
Wall time: 1min 6s


In [ ]:
# Pretty good... indexing implied_condition will likely bring this down even further, but memory
# becomes an issue for me at this point